In [1]:
from math import sqrt
import copy
import  traceback
import shutil

import numpy as np  # linear algebra
import pydicom
import os
import matplotlib.pyplot as plt
import cv2
from pydicom.uid import UID
from PIL import Image

In [2]:
def load_scan(path):
    slices = [pydicom.dcmread(path + '/' + s) for s in filter(lambda x: x.endswith('.dcm'), os.listdir(path))]
    slices.sort(key=lambda x: float(x.InstanceNumber))
    return slices

In [3]:
cta_root_path = '/nfs3-p1/zsxm/dataset/aorta_img_label/'
root_path = '/nfs3-p1/zsxm/dataset/aorta/'
input_folder_list = []
for path in os.listdir(root_path):
    if not os.path.isdir(os.path.join(root_path, path)):
        continue
    if os.path.exists(os.path.join(root_path, path, '1/')) and os.path.exists(os.path.join(root_path, path, '2/')):
        input_folder_list.append(os.path.join(root_path, path))
    else:
        print(path)

yujinmin-Im25-78


In [5]:
complete_path = []
ignore_path = []
for patient in input_folder_list:
    sl = patient.split('/')
    name = sl[-2] if sl[-1] == '' else sl[-1]
    print(f'Processing {name}')
    
    image_path = os.path.join(patient, '1', 'images')
    label_path = os.path.join(patient, '1', 'labels')
    if os.path.exists(image_path):
        shutil.rmtree(image_path)
    if os.path.exists(label_path):
        shutil.rmtree(label_path)
    os.mkdir(image_path)
    os.mkdir(label_path)
    with open(os.path.join(label_path, f'labels.txt'), 'w') as txt:
        txt.write('aorta')
    
    ct = load_scan(os.path.join(patient, '1'))
    cta = load_scan(os.path.join(patient, '2'))
    try:
        ct_head, ct_tail, cta_head, cta_tail = ct[0].ImagePositionPatient[2],ct[-1].ImagePositionPatient[2],cta[0].ImagePositionPatient[2],cta[-1].ImagePositionPatient[2]
    except:
        ct_head, ct_tail, cta_head, cta_tail = ct[0].SliceLocation,ct[-1].SliceLocation,cta[0].SliceLocation,cta[-1].SliceLocation
    if ct_head > cta_head or ct_tail < cta_tail:
        print(f'Ignore {name}')
        ignore_path.append(patient)
        shutil.rmtree(image_path)
        shutil.rmtree(label_path)
        continue
    
    p = 0
    for i in range(len(ct)):
        img = ct[i].pixel_array.astype(np.int16)
        intercept = ct[i].RescaleIntercept
        slope = ct[i].RescaleSlope
        if slope != 1:
            img = (slope * img.astype(np.float64)).astype(np.int16)
        img += np.int16(intercept)
        img = np.clip(img, -100, 500)
        img = ((img+100)/600*255).astype(np.uint8)
        img = Image.fromarray(img)
        img.save(os.path.join(image_path, f'{name}_{i:04d}.png'))
        with open(os.path.join(label_path, f'{name}_{i:04d}.txt'), 'w') as txt:
            while p+1 < len(cta):
                try:
                    ct_z, cta_p_z, cta_p1_z = ct[i].ImagePositionPatient[2], cta[p].ImagePositionPatient[2], cta[p+1].ImagePositionPatient[2]
                except:
                    ct_z, cta_p_z, cta_p1_z = ct[i].SliceLocation, cta[p].SliceLocation, cta[p+1].SliceLocation
                if (cta_p_z-ct_z) * (cta_p1_z-ct_z) <= 0:
                    p = p if abs(cta_p_z-ct_z) < abs(cta_p1_z-ct_z) else p+1
                    with open(os.path.join(cta_root_path, name, 'labels', f'{name}_{p:04d}.txt'), 'r') as sf:
                        txt.writelines(sf.readlines())
                    break
                p += 1
            else:
                raise Exception(f'CT图像范围超出了CTA图像范围：{ct_z},{cta_p_z},{cta_p1_z}')
    complete_path.append(patient)

Processing gengyongchao-Im21-83
Processing hebin-Im41-91
Ignore hebin-Im41-91
Processing hejianjun-Im31-99
Processing heshengnan-Im16-90
Processing huangjianhua-Im38-144
Processing hujiaguo-Im34-58
Processing jiangquanfeng-Im 50-107
Processing litingxian-Im36-141
Processing luohuafa-im29-96
Ignore luohuafa-im29-96
Processing lushida-Im33-159
Processing lvpeiji-Im32-136
Processing manyanfeng-Im32-153
Processing maoliang-Im12-90
Processing modehe-Im34-89
Processing nibingshan-Im33-42
Processing qiquanping-Im25-54
Processing shengfumei-Im31-132
Processing shenhuagou-Im33-95
Processing shenjiaren-Im35-136
Processing shenjie-Im36-100
Processing shiaihua-Im28-108
Ignore shiaihua-Im28-108
Processing shilei-Im21-92
Ignore shilei-Im21-92
Processing shuzhiming-Im28-80
Processing taofangguo-Im30-142
Processing wangaishui-Im75-128
Processing wanggangxiang-Im16-90
Processing wanggaosheng-Im24-57
Ignore wanggaosheng-Im24-57
Processing wangjianjun-Im30-110
Processing wanglianlian-Im42-139
Processing 

In [6]:
print(len(input_folder_list), len(complete_path), len(ignore_path))

64 53 11


In [8]:
for patient in ignore_path:
    sl = patient.split('/')
    name = sl[-2] if sl[-1] == '' else sl[-1]
    print(f'Processing {name}')
    
    image_path = os.path.join(patient, '1', 'images')
    if os.path.exists(image_path):
        shutil.rmtree(image_path)
    os.mkdir(image_path)
    
    ct = load_scan(os.path.join(patient, '1'))
    
    for i in range(len(ct)):
        img = ct[i].pixel_array.astype(np.int16)
        intercept = ct[i].RescaleIntercept
        slope = ct[i].RescaleSlope
        if slope != 1:
            img = (slope * img.astype(np.float64)).astype(np.int16)
        img += np.int16(intercept)
        img = np.clip(img, -100, 500)
        img = ((img+100)/600*255).astype(np.uint8)
        img = Image.fromarray(img)
        img.save(os.path.join(image_path, f'{name}_{i:04d}.png'))

Processing hebin-Im41-91
Processing luohuafa-im29-96
Processing shiaihua-Im28-108
Processing shilei-Im21-92
Processing wanggaosheng-Im24-57
Processing wangzhongwen-Im36-80
Processing yinghao-Im18-86
Processing chuguangzhou-Im46-111
Processing zhuyongbu-im23-92
Processing gengguorui-Im18-66
Processing chengxuejun-Im17-83


In [9]:
dst_path = '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/wrong'
if not os.path.exists(dst_path):
    os.mkdir(dst_path)
for p in ignore_path:
    sl = p.split('/')
    name = sl[-1] if sl[-1] != '' else sl[-2]
    print(f"Copy {os.path.join(p)} To {os.path.join(dst_path, name)}")
    if os.path.exists(os.path.join(dst_path, name)):
        print(f"remove {os.path.join(dst_path, name)}")
        shutil.rmtree(os.path.join(dst_path, name))
    os.mkdir(os.path.join(dst_path, name))
        
    try:
        shutil.copytree(os.path.join(p, '1', 'images'), os.path.join(dst_path, name, 'images'))
    except:
        traceback.print_exc()

Copy /nfs3-p1/zsxm/dataset/aorta/hebin-Im41-91 To /nfs3-p1/zsxm/dataset/aorta_ct_img_label/wrong/hebin-Im41-91
Copy /nfs3-p1/zsxm/dataset/aorta/luohuafa-im29-96 To /nfs3-p1/zsxm/dataset/aorta_ct_img_label/wrong/luohuafa-im29-96
Copy /nfs3-p1/zsxm/dataset/aorta/shiaihua-Im28-108 To /nfs3-p1/zsxm/dataset/aorta_ct_img_label/wrong/shiaihua-Im28-108
Copy /nfs3-p1/zsxm/dataset/aorta/shilei-Im21-92 To /nfs3-p1/zsxm/dataset/aorta_ct_img_label/wrong/shilei-Im21-92
Copy /nfs3-p1/zsxm/dataset/aorta/wanggaosheng-Im24-57 To /nfs3-p1/zsxm/dataset/aorta_ct_img_label/wrong/wanggaosheng-Im24-57
Copy /nfs3-p1/zsxm/dataset/aorta/wangzhongwen-Im36-80 To /nfs3-p1/zsxm/dataset/aorta_ct_img_label/wrong/wangzhongwen-Im36-80
Copy /nfs3-p1/zsxm/dataset/aorta/yinghao-Im18-86 To /nfs3-p1/zsxm/dataset/aorta_ct_img_label/wrong/yinghao-Im18-86
Copy /nfs3-p1/zsxm/dataset/aorta/chuguangzhou-Im46-111 To /nfs3-p1/zsxm/dataset/aorta_ct_img_label/wrong/chuguangzhou-Im46-111
Copy /nfs3-p1/zsxm/dataset/aorta/zhuyongbu-im23-

In [7]:
dst_path = '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/new'
if not os.path.exists(dst_path):
    os.mkdir(dst_path)
for p in complete_path:
    sl = p.split('/')
    name = sl[-1] if sl[-1] != '' else sl[-2]
    print(f"Copy {os.path.join(p)} To {os.path.join(dst_path, name)}")
    if os.path.exists(os.path.join(dst_path, name)):
        print(f"remove {os.path.join(dst_path, name)}")
        shutil.rmtree(os.path.join(dst_path, name))
    os.mkdir(os.path.join(dst_path, name))
        
    try:
        shutil.copytree(os.path.join(p, '1', 'images'), os.path.join(dst_path, name, 'images'))
        shutil.copytree(os.path.join(p, '1', 'labels'), os.path.join(dst_path, name, 'labels'))
    except:
        traceback.print_exc()

Copy /nfs3-p1/zsxm/dataset/aorta/gengyongchao-Im21-83 To /nfs3-p1/zsxm/dataset/aorta_ct_img_label/new/gengyongchao-Im21-83
Copy /nfs3-p1/zsxm/dataset/aorta/hejianjun-Im31-99 To /nfs3-p1/zsxm/dataset/aorta_ct_img_label/new/hejianjun-Im31-99
Copy /nfs3-p1/zsxm/dataset/aorta/heshengnan-Im16-90 To /nfs3-p1/zsxm/dataset/aorta_ct_img_label/new/heshengnan-Im16-90
Copy /nfs3-p1/zsxm/dataset/aorta/huangjianhua-Im38-144 To /nfs3-p1/zsxm/dataset/aorta_ct_img_label/new/huangjianhua-Im38-144
Copy /nfs3-p1/zsxm/dataset/aorta/hujiaguo-Im34-58 To /nfs3-p1/zsxm/dataset/aorta_ct_img_label/new/hujiaguo-Im34-58
Copy /nfs3-p1/zsxm/dataset/aorta/jiangquanfeng-Im 50-107 To /nfs3-p1/zsxm/dataset/aorta_ct_img_label/new/jiangquanfeng-Im 50-107
Copy /nfs3-p1/zsxm/dataset/aorta/litingxian-Im36-141 To /nfs3-p1/zsxm/dataset/aorta_ct_img_label/new/litingxian-Im36-141
Copy /nfs3-p1/zsxm/dataset/aorta/lushida-Im33-159 To /nfs3-p1/zsxm/dataset/aorta_ct_img_label/new/lushida-Im33-159
Copy /nfs3-p1/zsxm/dataset/aorta/lvp

In [10]:
def move_to_yolo(input_path, img_path, label_path):
    if not os.path.exists(img_path):
        os.makedirs(img_path)
    if not os.path.exists(label_path):
        os.makedirs(label_path)

    for patient in os.listdir(input_path):
        if not os.path.isdir(os.path.join(input_path, patient)):
            continue

        print(f'Processing {patient}')

        for f in os.listdir(os.path.join(input_path, patient, 'images')):
            shutil.copy(os.path.join(input_path, patient, 'images',f), img_path)

        for f in os.listdir(os.path.join(input_path, patient, 'labels')):
            if f == 'labels.txt':
                continue
            shutil.copy(os.path.join(input_path, patient, 'labels',f), label_path)

input_path = '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/train/'
img_path = '/nfs3-p1/zsxm/dataset/aorta_yolo_ct/images/train/'
label_path = '/nfs3-p1/zsxm/dataset/aorta_yolo_ct/labels_ori/train/'
move_to_yolo(input_path, img_path, label_path)
input_path = '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/val/'
img_path = '/nfs3-p1/zsxm/dataset/aorta_yolo_ct/images/val/'
label_path = '/nfs3-p1/zsxm/dataset/aorta_yolo_ct/labels_ori/val/'
move_to_yolo(input_path, img_path, label_path)

Processing dongxuesheng-Im37-149
Processing durenxi-Im57-172
Processing fangcaiyu-Im21-91
Processing fengyonghua-Im21-89
Processing fuhuajin-Im29-126
Processing gaomingsheng-Im34-151
Processing gengyongchao-Im21-83
Processing hejianjun-Im31-99
Processing heshengnan-Im16-90
Processing huangjianhua-Im38-144
Processing jiangquanfeng-Im 50-107
Processing litingxian-Im36-141
Processing lushida-Im33-159
Processing manyanfeng-Im32-153
Processing modehe-Im34-89
Processing nibingshan-Im33-42
Processing shenhuagou-Im33-95
Processing shenjie-Im36-100
Processing shuzhiming-Im28-80
Processing taofangguo-Im30-142
Processing wangaishui-Im75-128
Processing wanggangxiang-Im16-90
Processing wangjianjun-Im30-110
Processing wanglianlian-Im42-139
Processing wangmanying-Im23-134
Processing yangwenbin-Im44-125
Processing yaoliang-Im30-83
Processing yuanenchun-Im27-93
Processing yuliyao-Im25-31
Processing zengzhengyou-Im54-145
Processing zhangxinliang-Im16-24
Processing zhaojianbin-Im31-158
Processing zhoufei

In [11]:
def correct_to_square(input_path, output_path):
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    for txt in os.listdir(input_path):
        if not txt.endswith('.txt') or txt == 'labels.txt':
            continue

        with open(os.path.join(input_path, txt), 'r') as sf:
            lines = sf.readlines()
        assert len(lines) < 3
        with open(os.path.join(output_path, txt), 'w') as df:
            for i in range(len(lines)):
                aorta = lines[i].split()
                aorta_f = list(map(lambda x: float(x), aorta))
                max_edge = max(float(aorta[3]), float(aorta[4]))
                head = "" if i == 0 else "\n"
                df.write(f'{head}0 {aorta[1]} {aorta[2]} {max_edge:.6f} {max_edge:.6f}')
                
input_path = '/nfs3-p1/zsxm/dataset/aorta_yolo_ct/labels_ori/train/'
output_path = '/nfs3-p1/zsxm/dataset/aorta_yolo_ct/labels/train/'
correct_to_square(input_path, output_path)
input_path = '/nfs3-p1/zsxm/dataset/aorta_yolo_ct/labels_ori/val/'
output_path = '/nfs3-p1/zsxm/dataset/aorta_yolo_ct/labels/val/'
correct_to_square(input_path, output_path)

[[], [], [], [], [], [], [], [], [], []]


In [ ]:
#这种做法是错误的，会导致candidate_table[0] == candidate_table[1] 改变其中一个另一个也跟着改变#candidate_table = [[]] * 256
train_ct_path = '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/train/'
train_ct_list = os.listdir(train_ct_path)
for patient in input_folder_list:
    sl = patient.split('/')
    name = sl[-2] if sl[-1] == '' else sl[-1]
    if name not in train_ct_list:
        continue
    print(f'Processing {name}')
    
    ct = load_scan(os.path.join(patient, '1'))
    cta = load_scan(os.path.join(patient, '2'))
    try:
        ct_head, ct_tail, cta_head, cta_tail = ct[0].ImagePositionPatient[2],ct[-1].ImagePositionPatient[2],cta[0].ImagePositionPatient[2],cta[-1].ImagePositionPatient[2]
    except:
        ct_head, ct_tail, cta_head, cta_tail = ct[0].SliceLocation,ct[-1].SliceLocation,cta[0].SliceLocation,cta[-1].SliceLocation
    if ct_head > cta_head or ct_tail < cta_tail:
        print(f'Ignore {name}')
        continue
    
    p = 0
    for i in range(len(ct)):
        img = ct[i].pixel_array.astype(np.int16)
        intercept = ct[i].RescaleIntercept
        slope = ct[i].RescaleSlope
        if slope != 1:
            img = (slope * img.astype(np.float64)).astype(np.int16)
        img += np.int16(intercept)
        img = np.clip(img, -100, 500)
        img = ((img+100)/600*255).astype(np.uint8)
#         img = Image.fromarray(img)
#         img.save(os.path.join(patient, '1', 'images', f'{name}_{i:04d}.png'))
        while p+1 < len(cta):
            try:
                ct_z, cta_p_z, cta_p1_z = ct[i].ImagePositionPatient[2], cta[p].ImagePositionPatient[2], cta[p+1].ImagePositionPatient[2]
            except:
                ct_z, cta_p_z, cta_p1_z = ct[i].SliceLocation, cta[p].SliceLocation, cta[p+1].SliceLocation
            if (cta_p_z-ct_z) * (cta_p1_z-ct_z) <= 0:
                p = p if abs(cta_p_z-ct_z) < abs(cta_p1_z-ct_z) else p+1
                with open(os.path.join(cta_root_path, name, 'labels', f'{name}_{p:04d}.txt'), 'r') as sf:
                    cta_labels = sf.readlines()
                cta_img = cta[p].pixel_array.astype(np.int16)
                cta_intercept = cta[p].RescaleIntercept
                cta_slope = cta[p].RescaleSlope
                if cta_slope != 1:
                    cta_img = (cta_slope * cta_img.astype(np.float64)).astype(np.int16)
                cta_img += np.int16(cta_intercept)
                cta_img = np.clip(cta_img, -100, 500)
                cta_img = ((cta_img+100)/600*255).astype(np.uint8)
                break
            p += 1
        else:
            raise Exception(f'CT图像范围超出了CTA图像范围：{ct_z},{cta_p_z},{cta_p1_z}')

In [56]:
val_label_path = '/nfs3-p1/zsxm/dataset/aorta_yolo_ct/labels/val/'
result_label_path = '/nfs3-p1/zsxm/dataset/aorta_yolo_ct/pred_labels/'

branch_count, branch_corr, other_count, other_corr = 0,0,0,0

def iou(pred, target):
    sp, st = pred[2]*pred[3], target[2]*target[3]
    px1, py1, px2, py2 = pred[0]-pred[2]/2, pred[1]-pred[3]/2, pred[0]+pred[2]/2, pred[1]+pred[3]/2
    tx1, ty1, tx2, ty2 = target[0]-target[2]/2, target[1]-target[3]/2, target[0]+target[2]/2, target[1]+target[3]/2
    ix1, iy1, ix2, iy2 = max(px1, tx1), max(py1, ty1), min(px2, tx2), min(py2, ty2)
    if ix1>ix2 or iy1>iy2:
        return 0.0
    si = max((ix2-ix1)*(iy2-iy1), 0)
    i_o_u = si/(sp+st-si)
    assert 0.0<=i_o_u<=1.0
    return i_o_u

for label in os.listdir(val_label_path):
    with open(os.path.join(val_label_path, label), 'r') as vf:
        val_lines = vf.readlines()
    assert 0<=len(val_lines)<=2
    if len(val_lines) == 0:
        continue
        
    if len(val_lines) == 1:
        other_count += 1
        tx, ty, tw, th = list(map(lambda x: float(x), val_lines[0].split()))[1:]
        try:
            with open(os.path.join(result_label_path, label), 'r') as rf:
                result_lines = rf.readlines()
        except Exception as e:
            print(e)
            continue
        for i in range(len(result_lines)):
            px, py, pw, ph = list(map(lambda x: float(x), result_lines[i].split()))[1:]
            if iou((px, py, pw, ph), (tx, ty, tw, th)) > 0.5:
                other_corr += 1
                break
    else:
        other_count += 1
        branch_count += 1
        line0, line1 = val_lines[0].split(), val_lines[1].split()
        line0_f, line1_f = list(map(lambda x: float(x), line0)), list(map(lambda x: float(x), line1))
        if line0_f[2] > line1_f[2]:
            aorta, branch = line0_f, line1_f
        else:
            aorta, branch = line1_f, line0_f
        atx, aty, atw, ath = aorta[1:]
        btx, bty, btw, bth = branch[1:]
        try:
            with open(os.path.join(result_label_path, label), 'r') as rf:
                result_lines = rf.readlines()
        except:
            continue
        for i in range(len(result_lines)):
            px, py, pw, ph = list(map(lambda x: float(x), result_lines[i].split()))[1:]
            if iou((px, py, pw, ph), (atx, aty, atw, ath)) > 0.5:
                other_corr += 1
                break
        for i in range(len(result_lines)):
            px, py, pw, ph = list(map(lambda x: float(x), result_lines[i].split()))[1:]
            if iou((px, py, pw, ph), (btx, bty, btw, bth)) > 0.5:
                branch_corr += 1
                break
                
print(f'branch recall:{branch_corr/branch_count}, other recall:{other_corr/other_count}')

[Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_yolo_ct/pred_labels/hujiaguo-Im34-58_0019.txt'
[Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_yolo_ct/pred_labels/hujiaguo-Im34-58_0020.txt'
[Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_yolo_ct/pred_labels/shengfumei-Im31-132_0020.txt'
[Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_yolo_ct/pred_labels/shengfumei-Im31-132_0122.txt'
[Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_yolo_ct/pred_labels/shengfumei-Im31-132_0123.txt'
[Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_yolo_ct/pred_labels/shengfumei-Im31-132_0125.txt'
[Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_yolo_ct/pred_labels/shengfumei-Im31-132_0126.txt'
[Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_yolo_ct/pred_labels/shengfumei-Im31-132_0127.txt'
[Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_yolo_ct/pred